# KUIS 2 Pembelajaran Mesin

## Tentang dataset

Pada sebuah lembaga kemanusiaan berbasis teknologi informasi, dibentuk beberapa tim kemanusiaan yang bertugas untuk terjun ke masyarakat memberikan bantuan dalam bentuk aplikasi komputer atau layanan pelatihan IT.  Kegiatn tersebut dilaksanakan setiap tahun dengan ketentuan anggota organisasi tersebut dapat bergabung ke beberapa tim sebagai ketua atau anggota. Terdapat pula anggota yang sudah tidak aktif di organisasi tersebut (terhapus di database mereka —> label N/A).  Organisasi tersebut juga melakukan rekrutmen volunteer, namun tidak semua tim menggunakan bantuan volunteer. Hasil kegiatan dari setiap tim dapat diwujudkan ke dalam sebuah pengakuan hak cipta atau publikasi karya ilmiah atau publikasi berita di media massa.

## Ketentuan
1. Pengerjaan secara individu alias setiap mahasiswa!
2. Terdapat data dengan nilai #N/A, pertimbangkan cara terbaik untuk mengatasi nilai tersebut.
3. Tidak boleh ada duplikasi pekerjaan antar mahasiswa
4. Berikan penjelasan untuk setiap tahapan. Jangan hanya kode!

## Soal
1. Lakukan pemilahan data training dan testing yang sesuai dan proporsional dari data tersebut! (10 poin)
2. Tentukan data validasi! (10 poin)
3. Tentukan secara tepat kolom yang menjadi "label" (20 poin)
4. Terapkan dua jenis metode klasifikasi dan lakukan perbandingan akurasi  dengan wajib melibatkan variabel keterlibatan volunteer sampai variabel luaran-luaran yang dihasilkan! (20 poin)
5. Lakukan tunning hyperparameter berdasarkan hasil klasifikasi pada nomor 3 menggunakan metode Grid Search dan Random Search! Simpulkan parameter apa yang paling baik digunakan berdasarkan metode klasifikasi yang dipilih, dan berapa akurasinya. (40 poin)

In [94]:
import pandas as pd

df = pd.read_excel('data/dataset_quiz.xlsx')

df.head(100)

,id_ketua,id_anggota1,id_anggota2,id_anggota3,id_anggota4,id_anggota5,keterlibatan_vltr,jml_vltr_terlibat,tahun,skema,luaran_hki,jenis_luaran_hki,potensi_hki,luaran_lain,jenis_luaran_lain,status_luaran_lain
0,7206.0,7214.0,6211.0,8602.0,7801.0,NaN,TIDAK,0,2019,REGULER,TIDAK ADA,TIDAK ADA,TIDAK,TIDAK ADA,TIDAK ADA,TIDAK ADA
1,7214.0,7109.0,8402.0,7903.0,9202.0,NaN,TIDAK,0,2019,REGULER,TIDAK ADA,TIDAK ADA,TIDAK,TIDAK ADA,TIDAK ADA,TIDAK ADA
2,5911.0,8201.0,8701.0,9101.0,9003.0,NaN,TIDAK,0,2019,REGULER,TIDAK ADA,TIDAK ADA,TIDAK,TIDAK ADA,TIDAK ADA,TIDAK ADA
3,8402.0,7702.0,8602.0,8903.0,8903.0,NaN,TIDAK,0,2019,REGULER,TIDAK ADA,TIDAK ADA,TIDAK,TIDAK ADA,TIDAK ADA,TIDAK ADA
4,8201.0,8305.0,5911.0,5903.0,8402.0,NaN,TIDAK,0,2019,REGULER,TIDAK ADA,TIDAK ADA,TIDAK,TIDAK ADA,TIDAK ADA,TIDAK ADA
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
80,8803.0,8804.0,9102.0,8005.0,9303.0,NaN,YA,3,2022,REGULER,TIDAK ADA,TIDAK ADA,TIDAK,TIDAK ADA,TIDAK ADA,TIDAK ADA
81,6211.0,8901.0,9003.0,9303.0,9007.0,NaN,YA,3,2022,REGULER,TIDAK ADA,TIDAK ADA,TIDAK,TIDAK ADA,TIDAK ADA,TIDAK ADA
82,8601.0,8903.0,8703.0,NaN,NaN,NaN,YA,3,2022,REGULER,TIDAK ADA,TIDAK ADA,TIDAK,TIDAK ADA,TIDAK ADA,TIDAK ADA
83,8703.0,8601.0,9202.0,9102.0,6204.0,NaN,YA,3,2022,REGULER,TIDAK ADA,TIDAK ADA,TIDAK,TIDAK ADA,TIDAK ADA,TIDAK ADA


## Validasi Data

Menentukan data validasi

In [95]:
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

In [96]:
df.shape

(85, 16)

In [97]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 85 entries, 0 to 84
Data columns (total 16 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   id_ketua            82 non-null     float64
 1   id_anggota1         80 non-null     float64
 2   id_anggota2         81 non-null     float64
 3   id_anggota3         82 non-null     float64
 4   id_anggota4         72 non-null     float64
 5   id_anggota5         1 non-null      float64
 6   keterlibatan_vltr   85 non-null     object 
 7   jml_vltr_terlibat   85 non-null     int64  
 8   tahun               85 non-null     int64  
 9   skema               85 non-null     object 
 10  luaran_hki          85 non-null     object 
 11  jenis_luaran_hki    85 non-null     object 
 12  potensi_hki         85 non-null     object 
 13  luaran_lain         85 non-null     object 
 14  jenis_luaran_lain   85 non-null     object 
 15  status_luaran_lain  85 non-null     object 
dtypes: float64

In [98]:
categorical = [var for var in df.columns if df[var].dtype=='O']

print('There are {} categorical variables\n'.format(len(categorical)))

print('The categorical variables are :\n\n', categorical)

There are 8 categorical variables

The categorical variables are :

 ['keterlibatan_vltr', 'skema', 'luaran_hki', 'jenis_luaran_hki', 'potensi_hki', 'luaran_lain', 'jenis_luaran_lain', 'status_luaran_lain']


In [99]:
df[categorical].isnull().sum()

keterlibatan_vltr     0
skema                 0
luaran_hki            0
jenis_luaran_hki      0
potensi_hki           0
luaran_lain           0
jenis_luaran_lain     0
status_luaran_lain    0
dtype: int64

In [100]:
for var in categorical: 
    
    print(df[var].value_counts())

TIDAK    45
YA       40
Name: keterlibatan_vltr, dtype: int64
REGULER      80
KEMITRAAN     5
Name: skema, dtype: int64
TIDAK ADA    77
ADA           8
Name: luaran_hki, dtype: int64
TIDAK ADA    77
Hak Cipta     8
Name: jenis_luaran_hki, dtype: int64
TIDAK    70
YA        8
SUDAH     6
DRAFT     1
Name: potensi_hki, dtype: int64
TIDAK ADA    69
YA           16
Name: luaran_lain, dtype: int64
TIDAK ADA                69
Artikel Ilmiah           10
Publikasi Media Massa     5
Modul Ajar                1
Name: jenis_luaran_lain, dtype: int64
TIDAK ADA       70
Terpublikasi    13
Draft            2
Name: status_luaran_lain, dtype: int64


In [101]:
for var in categorical: 
    
    print(df[var].value_counts()/np.float(len(df)))

TIDAK    0.529412
YA       0.470588
Name: keterlibatan_vltr, dtype: float64
REGULER      0.941176
KEMITRAAN    0.058824
Name: skema, dtype: float64
TIDAK ADA    0.905882
ADA          0.094118
Name: luaran_hki, dtype: float64
TIDAK ADA    0.905882
Hak Cipta    0.094118
Name: jenis_luaran_hki, dtype: float64
TIDAK    0.823529
YA       0.094118
SUDAH    0.070588
DRAFT    0.011765
Name: potensi_hki, dtype: float64
TIDAK ADA    0.811765
YA           0.188235
Name: luaran_lain, dtype: float64
TIDAK ADA                0.811765
Artikel Ilmiah           0.117647
Publikasi Media Massa    0.058824
Modul Ajar               0.011765
Name: jenis_luaran_lain, dtype: float64
TIDAK ADA       0.823529
Terpublikasi    0.152941
Draft           0.023529
Name: status_luaran_lain, dtype: float64


C:\Users\ADMINI~1\AppData\Local\Temp/ipykernel_9416/338286487.py:3: DeprecationWarning: `np.float` is a deprecated alias for the builtin `float`. To silence this warning, use `float` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.float64` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  print(df[var].value_counts()/np.float(len(df)))


In [102]:
df.keterlibatan_vltr.value_counts()

TIDAK    45
YA       40
Name: keterlibatan_vltr, dtype: int64

In [103]:
df.skema.value_counts()

REGULER      80
KEMITRAAN     5
Name: skema, dtype: int64

In [104]:
df.luaran_hki.value_counts()

TIDAK ADA    77
ADA           8
Name: luaran_hki, dtype: int64

In [105]:
df.jenis_luaran_hki.value_counts()

TIDAK ADA    77
Hak Cipta     8
Name: jenis_luaran_hki, dtype: int64

In [106]:
df.potensi_hki.value_counts()

TIDAK    70
YA        8
SUDAH     6
DRAFT     1
Name: potensi_hki, dtype: int64

In [107]:
df.luaran_lain.value_counts()

TIDAK ADA    69
YA           16
Name: luaran_lain, dtype: int64

In [108]:
df.jenis_luaran_lain.value_counts()

TIDAK ADA                69
Artikel Ilmiah           10
Publikasi Media Massa     5
Modul Ajar                1
Name: jenis_luaran_lain, dtype: int64

In [109]:
df.status_luaran_lain.value_counts()

TIDAK ADA       70
Terpublikasi    13
Draft            2
Name: status_luaran_lain, dtype: int64

In [110]:
df[categorical].isnull().sum()

keterlibatan_vltr     0
skema                 0
luaran_hki            0
jenis_luaran_hki      0
potensi_hki           0
luaran_lain           0
jenis_luaran_lain     0
status_luaran_lain    0
dtype: int64

In [111]:
numerical = [var for var in df.columns if df[var].dtype!='O']

print('There are {} numerical variables\n'.format(len(numerical)))

print('The numerical variables are :', numerical)

There are 8 numerical variables

The numerical variables are : ['id_ketua', 'id_anggota1', 'id_anggota2', 'id_anggota3', 'id_anggota4', 'id_anggota5', 'jml_vltr_terlibat', 'tahun']


In [112]:
df[numerical].isnull().sum()

id_ketua              3
id_anggota1           5
id_anggota2           4
id_anggota3           3
id_anggota4          13
id_anggota5          84
jml_vltr_terlibat     0
tahun                 0
dtype: int64

## Label encoder

In [113]:
labeled_data = df

labeled_data

,id_ketua,id_anggota1,id_anggota2,id_anggota3,id_anggota4,id_anggota5,keterlibatan_vltr,jml_vltr_terlibat,tahun,skema,luaran_hki,jenis_luaran_hki,potensi_hki,luaran_lain,jenis_luaran_lain,status_luaran_lain
0,7206.0,7214.0,6211.0,8602.0,7801.0,NaN,TIDAK,0,2019,REGULER,TIDAK ADA,TIDAK ADA,TIDAK,TIDAK ADA,TIDAK ADA,TIDAK ADA
1,7214.0,7109.0,8402.0,7903.0,9202.0,NaN,TIDAK,0,2019,REGULER,TIDAK ADA,TIDAK ADA,TIDAK,TIDAK ADA,TIDAK ADA,TIDAK ADA
2,5911.0,8201.0,8701.0,9101.0,9003.0,NaN,TIDAK,0,2019,REGULER,TIDAK ADA,TIDAK ADA,TIDAK,TIDAK ADA,TIDAK ADA,TIDAK ADA
3,8402.0,7702.0,8602.0,8903.0,8903.0,NaN,TIDAK,0,2019,REGULER,TIDAK ADA,TIDAK ADA,TIDAK,TIDAK ADA,TIDAK ADA,TIDAK ADA
4,8201.0,8305.0,5911.0,5903.0,8402.0,NaN,TIDAK,0,2019,REGULER,TIDAK ADA,TIDAK ADA,TIDAK,TIDAK ADA,TIDAK ADA,TIDAK ADA
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
80,8803.0,8804.0,9102.0,8005.0,9303.0,NaN,YA,3,2022,REGULER,TIDAK ADA,TIDAK ADA,TIDAK,TIDAK ADA,TIDAK ADA,TIDAK ADA
81,6211.0,8901.0,9003.0,9303.0,9007.0,NaN,YA,3,2022,REGULER,TIDAK ADA,TIDAK ADA,TIDAK,TIDAK ADA,TIDAK ADA,TIDAK ADA
82,8601.0,8903.0,8703.0,NaN,NaN,NaN,YA,3,2022,REGULER,TIDAK ADA,TIDAK ADA,TIDAK,TIDAK ADA,TIDAK ADA,TIDAK ADA
83,8703.0,8601.0,9202.0,9102.0,6204.0,NaN,YA,3,2022,REGULER,TIDAK ADA,TIDAK ADA,TIDAK,TIDAK ADA,TIDAK ADA,TIDAK ADA


## Memilih Label

Menentukan kolom yang menjadi "label"

In [114]:
# Pisahkan label dan fitur
X = labeled_data.drop(['keterlibatan_vltr', 'id_anggota5'], axis=1)

y = labeled_data['keterlibatan_vltr']
X

,id_ketua,id_anggota1,id_anggota2,id_anggota3,id_anggota4,jml_vltr_terlibat,tahun,skema,luaran_hki,jenis_luaran_hki,potensi_hki,luaran_lain,jenis_luaran_lain,status_luaran_lain
0,7206.0,7214.0,6211.0,8602.0,7801.0,0,2019,REGULER,TIDAK ADA,TIDAK ADA,TIDAK,TIDAK ADA,TIDAK ADA,TIDAK ADA
1,7214.0,7109.0,8402.0,7903.0,9202.0,0,2019,REGULER,TIDAK ADA,TIDAK ADA,TIDAK,TIDAK ADA,TIDAK ADA,TIDAK ADA
2,5911.0,8201.0,8701.0,9101.0,9003.0,0,2019,REGULER,TIDAK ADA,TIDAK ADA,TIDAK,TIDAK ADA,TIDAK ADA,TIDAK ADA
3,8402.0,7702.0,8602.0,8903.0,8903.0,0,2019,REGULER,TIDAK ADA,TIDAK ADA,TIDAK,TIDAK ADA,TIDAK ADA,TIDAK ADA
4,8201.0,8305.0,5911.0,5903.0,8402.0,0,2019,REGULER,TIDAK ADA,TIDAK ADA,TIDAK,TIDAK ADA,TIDAK ADA,TIDAK ADA
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
80,8803.0,8804.0,9102.0,8005.0,9303.0,3,2022,REGULER,TIDAK ADA,TIDAK ADA,TIDAK,TIDAK ADA,TIDAK ADA,TIDAK ADA
81,6211.0,8901.0,9003.0,9303.0,9007.0,3,2022,REGULER,TIDAK ADA,TIDAK ADA,TIDAK,TIDAK ADA,TIDAK ADA,TIDAK ADA
82,8601.0,8903.0,8703.0,NaN,NaN,3,2022,REGULER,TIDAK ADA,TIDAK ADA,TIDAK,TIDAK ADA,TIDAK ADA,TIDAK ADA
83,8703.0,8601.0,9202.0,9102.0,6204.0,3,2022,REGULER,TIDAK ADA,TIDAK ADA,TIDAK,TIDAK ADA,TIDAK ADA,TIDAK ADA


In [115]:
# Encode Fitur
from sklearn.preprocessing import LabelEncoder

def feature_encode(df):
    encode = LabelEncoder()
    for col in df:
        df[col] = encode.fit_transform(df[col])
    
    return df

X = feature_encode(X)
display(X)

# Encode Label
encode = LabelEncoder()
y = encode.fit_transform(y)
print(y)

,id_ketua,id_anggota1,id_anggota2,id_anggota3,id_anggota4,jml_vltr_terlibat,tahun,skema,luaran_hki,jenis_luaran_hki,potensi_hki,luaran_lain,jenis_luaran_lain,status_luaran_lain
0,4,6,3,18,4,0,0,1,1,1,2,0,3,1
1,5,4,14,8,31,0,0,1,1,1,2,0,3,1
2,1,15,18,37,25,0,0,1,1,1,2,0,3,1
3,20,10,16,30,20,0,0,1,1,1,2,0,3,1
4,17,17,2,1,13,0,0,1,1,1,2,0,3,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
80,26,26,35,10,33,3,3,1,1,1,2,0,3,1
81,3,28,33,43,26,3,3,1,1,1,2,0,3,1
82,21,29,20,44,34,3,3,1,1,1,2,0,3,1
83,25,19,39,38,1,3,3,1,1,1,2,0,3,1


[0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 1 1 1 0 1 1 1 0 1 1 0 1 1 1 0 1
 0 1 0 1 1 0 0 1 1 1 1 1 1 0 0 0 1 1 0 0 0 0 0 1 1 1 1 1 0 0 0 1 0 0 0 1 0
 1 0 1 1 1 1 1 1 1 1 0]


## Data Training & Testing

Pemilahan data training dan testing

In [116]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.3, random_state = 0)

In [117]:
X_train.shape, X_test.shape

((59, 14), (26, 14))

In [118]:
X_train.dtypes

id_ketua              int64
id_anggota1           int64
id_anggota2           int64
id_anggota3           int64
id_anggota4           int64
jml_vltr_terlibat     int64
tahun                 int64
skema                 int32
luaran_hki            int32
jenis_luaran_hki      int32
potensi_hki           int32
luaran_lain           int32
jenis_luaran_lain     int32
status_luaran_lain    int32
dtype: object

In [119]:

cols = X_train.columns

In [120]:
from sklearn.preprocessing import RobustScaler

scaler = RobustScaler()

X_train = scaler.fit_transform(X_train)

X_test = scaler.transform(X_test)

In [121]:
X_train = pd.DataFrame(X_train, columns=[cols])

In [122]:
X_test = pd.DataFrame(X_test, columns=[cols])

In [123]:
X_train.head()

,id_ketua,id_anggota1,id_anggota2,id_anggota3,id_anggota4,jml_vltr_terlibat,tahun,skema,luaran_hki,jenis_luaran_hki,potensi_hki,luaran_lain,jenis_luaran_lain,status_luaran_lain
0,1.071429,0.666667,0.636364,0.227273,-0.294118,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.714286,-0.266667,-0.045455,-0.636364,0.235294,0.0,1.0,0.0,-1.0,-1.0,-1.0,1.0,-1.0,1.0
2,0.785714,-0.488889,-1.045455,0.681818,0.647059,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.214286,-0.355556,-0.318182,0.272727,-0.176471,0.0,-0.5,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,-0.285714,-0.400000,-0.681818,0.363636,0.588235,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [124]:
X_test.head()

,id_ketua,id_anggota1,id_anggota2,id_anggota3,id_anggota4,jml_vltr_terlibat,tahun,skema,luaran_hki,jenis_luaran_hki,potensi_hki,luaran_lain,jenis_luaran_lain,status_luaran_lain
0,-1.142857,-0.133333,-0.227273,0.590909,0.117647,0.000000,-0.5,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,-0.142857,0.488889,-0.363636,-0.090909,0.235294,0.000000,-0.5,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,-0.642857,-0.400000,-0.681818,-0.772727,0.647059,1.000000,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.785714,0.888889,0.227273,0.772727,0.411765,0.666667,0.5,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,-0.857143,0.577778,-1.045455,0.409091,-0.235294,1.000000,0.5,0.0,0.0,0.0,0.0,0.0,0.0,0.0


## Klasifikasi Naive Bayes

In [125]:
from sklearn.naive_bayes import GaussianNB
gnb = GaussianNB()
gnb.fit(X_train, y_train)

c:\Users\Administrator\anaconda3\lib\site-packages\sklearn\utils\validation.py:1688: FutureWarning: Feature names only support names that are all strings. Got feature names with dtypes: ['tuple']. An error will be raised in 1.2.
  warnings.warn(


GaussianNB()

In [126]:
y_pred = gnb.predict(X_test)

y_pred

c:\Users\Administrator\anaconda3\lib\site-packages\sklearn\utils\validation.py:1688: FutureWarning: Feature names only support names that are all strings. Got feature names with dtypes: ['tuple']. An error will be raised in 1.2.
  warnings.warn(


array([0, 0, 1, 1, 1, 1, 0, 1, 0, 0, 0, 1, 1, 1, 0, 0, 0, 1, 1, 1, 1, 0,
       0, 0, 1, 0])

In [127]:
from sklearn.metrics import accuracy_score

print('Model accuracy score: {0:0.4f}'. format(accuracy_score(y_test, y_pred)))

Model accuracy score: 1.0000


In [128]:
print('Training-set accuracy score: {0:0.4f}'. format(accuracy_score(y_train, y_pred_train)))

Training-set accuracy score: 0.9831


In [129]:
print('Training set score: {:.4f}'.format(gnb.score(X_train, y_train)))

print('Test set score: {:.4f}'.format(gnb.score(X_test, y_test)))

Training set score: 0.9831
Test set score: 1.0000


c:\Users\Administrator\anaconda3\lib\site-packages\sklearn\utils\validation.py:1688: FutureWarning: Feature names only support names that are all strings. Got feature names with dtypes: ['tuple']. An error will be raised in 1.2.
  warnings.warn(
c:\Users\Administrator\anaconda3\lib\site-packages\sklearn\utils\validation.py:1688: FutureWarning: Feature names only support names that are all strings. Got feature names with dtypes: ['tuple']. An error will be raised in 1.2.
  warnings.warn(


## Klasifikasi Decision Tree

In [130]:
import numpy as np
import pandas as pd
from sklearn.tree import DecisionTreeClassifier
from sklearn import tree
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

# Split train test
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=50)

# Initiate Model
dt_entropy = DecisionTreeClassifier(criterion='entropy')
dt_gini = DecisionTreeClassifier(criterion='gini')

# Fit model
# Entropy
dt_entropy.fit(X_train, y_train)
y_pred_entropy_train = dt_entropy.predict(X_train)
y_pred_entropy = dt_entropy.predict(X_test)

# Gini
dt_gini.fit(X_train, y_train)
y_pred_gini_train = dt_gini.predict(X_train)
y_pred_gini = dt_gini.predict(X_test)

# Evaluasi
# Entropy
acc_entropy_train = accuracy_score(y_train, y_pred_entropy_train)
acc_entropy = accuracy_score(y_test, y_pred_entropy)

# Gini
acc_gini_train = accuracy_score(y_train, y_pred_gini_train)
acc_gini = accuracy_score(y_test, y_pred_gini)

print(f'Akurasi Entropy Train: {acc_entropy_train}')
print(f'Akurasi Entropy: {acc_entropy}')
print('\n')
print(f'Akurasi Gini Train: {acc_gini_train}')
print(f'Akurasi Gini: {acc_gini}')

Akurasi Entropy Train: 1.0
Akurasi Entropy: 1.0


Akurasi Gini Train: 1.0
Akurasi Gini: 1.0


## Grid Search

In [131]:
from sklearn.model_selection import RepeatedKFold

# inisiasi repated k-fold
cv = RepeatedKFold(n_splits=4, n_repeats=3, random_state=42)

In [132]:
labeled_data = df

labeled_data

,id_ketua,id_anggota1,id_anggota2,id_anggota3,id_anggota4,id_anggota5,keterlibatan_vltr,jml_vltr_terlibat,tahun,skema,luaran_hki,jenis_luaran_hki,potensi_hki,luaran_lain,jenis_luaran_lain,status_luaran_lain
0,7206.0,7214.0,6211.0,8602.0,7801.0,NaN,TIDAK,0,2019,REGULER,TIDAK ADA,TIDAK ADA,TIDAK,TIDAK ADA,TIDAK ADA,TIDAK ADA
1,7214.0,7109.0,8402.0,7903.0,9202.0,NaN,TIDAK,0,2019,REGULER,TIDAK ADA,TIDAK ADA,TIDAK,TIDAK ADA,TIDAK ADA,TIDAK ADA
2,5911.0,8201.0,8701.0,9101.0,9003.0,NaN,TIDAK,0,2019,REGULER,TIDAK ADA,TIDAK ADA,TIDAK,TIDAK ADA,TIDAK ADA,TIDAK ADA
3,8402.0,7702.0,8602.0,8903.0,8903.0,NaN,TIDAK,0,2019,REGULER,TIDAK ADA,TIDAK ADA,TIDAK,TIDAK ADA,TIDAK ADA,TIDAK ADA
4,8201.0,8305.0,5911.0,5903.0,8402.0,NaN,TIDAK,0,2019,REGULER,TIDAK ADA,TIDAK ADA,TIDAK,TIDAK ADA,TIDAK ADA,TIDAK ADA
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
80,8803.0,8804.0,9102.0,8005.0,9303.0,NaN,YA,3,2022,REGULER,TIDAK ADA,TIDAK ADA,TIDAK,TIDAK ADA,TIDAK ADA,TIDAK ADA
81,6211.0,8901.0,9003.0,9303.0,9007.0,NaN,YA,3,2022,REGULER,TIDAK ADA,TIDAK ADA,TIDAK,TIDAK ADA,TIDAK ADA,TIDAK ADA
82,8601.0,8903.0,8703.0,NaN,NaN,NaN,YA,3,2022,REGULER,TIDAK ADA,TIDAK ADA,TIDAK,TIDAK ADA,TIDAK ADA,TIDAK ADA
83,8703.0,8601.0,9202.0,9102.0,6204.0,NaN,YA,3,2022,REGULER,TIDAK ADA,TIDAK ADA,TIDAK,TIDAK ADA,TIDAK ADA,TIDAK ADA


In [133]:
# Pisahkan label dan fitur
X = labeled_data.drop(['keterlibatan_vltr', 'id_anggota5'], axis=1)

y = labeled_data['keterlibatan_vltr']
X

,id_ketua,id_anggota1,id_anggota2,id_anggota3,id_anggota4,jml_vltr_terlibat,tahun,skema,luaran_hki,jenis_luaran_hki,potensi_hki,luaran_lain,jenis_luaran_lain,status_luaran_lain
0,7206.0,7214.0,6211.0,8602.0,7801.0,0,2019,REGULER,TIDAK ADA,TIDAK ADA,TIDAK,TIDAK ADA,TIDAK ADA,TIDAK ADA
1,7214.0,7109.0,8402.0,7903.0,9202.0,0,2019,REGULER,TIDAK ADA,TIDAK ADA,TIDAK,TIDAK ADA,TIDAK ADA,TIDAK ADA
2,5911.0,8201.0,8701.0,9101.0,9003.0,0,2019,REGULER,TIDAK ADA,TIDAK ADA,TIDAK,TIDAK ADA,TIDAK ADA,TIDAK ADA
3,8402.0,7702.0,8602.0,8903.0,8903.0,0,2019,REGULER,TIDAK ADA,TIDAK ADA,TIDAK,TIDAK ADA,TIDAK ADA,TIDAK ADA
4,8201.0,8305.0,5911.0,5903.0,8402.0,0,2019,REGULER,TIDAK ADA,TIDAK ADA,TIDAK,TIDAK ADA,TIDAK ADA,TIDAK ADA
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
80,8803.0,8804.0,9102.0,8005.0,9303.0,3,2022,REGULER,TIDAK ADA,TIDAK ADA,TIDAK,TIDAK ADA,TIDAK ADA,TIDAK ADA
81,6211.0,8901.0,9003.0,9303.0,9007.0,3,2022,REGULER,TIDAK ADA,TIDAK ADA,TIDAK,TIDAK ADA,TIDAK ADA,TIDAK ADA
82,8601.0,8903.0,8703.0,NaN,NaN,3,2022,REGULER,TIDAK ADA,TIDAK ADA,TIDAK,TIDAK ADA,TIDAK ADA,TIDAK ADA
83,8703.0,8601.0,9202.0,9102.0,6204.0,3,2022,REGULER,TIDAK ADA,TIDAK ADA,TIDAK,TIDAK ADA,TIDAK ADA,TIDAK ADA


In [134]:
# Encode Fitur
from sklearn.preprocessing import LabelEncoder

def feature_encode(df):
    encode = LabelEncoder()
    for col in df:
        df[col] = encode.fit_transform(df[col])
    
    return df

X = feature_encode(X)
display(X)

# Encode Label
encode = LabelEncoder()
y = encode.fit_transform(y)
print(y)

,id_ketua,id_anggota1,id_anggota2,id_anggota3,id_anggota4,jml_vltr_terlibat,tahun,skema,luaran_hki,jenis_luaran_hki,potensi_hki,luaran_lain,jenis_luaran_lain,status_luaran_lain
0,4,6,3,18,4,0,0,1,1,1,2,0,3,1
1,5,4,14,8,31,0,0,1,1,1,2,0,3,1
2,1,15,18,37,25,0,0,1,1,1,2,0,3,1
3,20,10,16,30,20,0,0,1,1,1,2,0,3,1
4,17,17,2,1,13,0,0,1,1,1,2,0,3,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
80,26,26,35,10,33,3,3,1,1,1,2,0,3,1
81,3,28,33,43,26,3,3,1,1,1,2,0,3,1
82,21,29,20,44,34,3,3,1,1,1,2,0,3,1
83,25,19,39,38,1,3,3,1,1,1,2,0,3,1


[0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 1 1 1 0 1 1 1 0 1 1 0 1 1 1 0 1
 0 1 0 1 1 0 0 1 1 1 1 1 1 0 0 0 1 1 0 0 0 0 0 1 1 1 1 1 0 0 0 1 0 0 0 1 0
 1 0 1 1 1 1 1 1 1 1 0]


In [135]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import GridSearchCV

# inisiasi model
dt = DecisionTreeClassifier()

# Definisikan hyperparameter yang akan digunakan
# sklearn menerima dalam bentuk dictionary
# nama hyperparamater HARUS SESUAI dengan dokumentasi sklearn
params = {
    'criterion': ['gini', 'entropy', 'log_loss'],
    'max_depth': list(range(5,11))
}

# inisiasi grid berdasarkan nilai repeated k-fold dan hyperparameter
grid = GridSearchCV(dt, param_grid=params, cv=cv)

# Fit / latih berdasarkan grid
# %timeit merupakan magic command didalam ipython notebook
# yang dapat kita gunakan untuk menghitung waktu komputasi
# cara ini cukup efektif untuk melakukan evaluasi suatu algoritma atau prosedur
%timeit grid.fit(X, y)

# Evaluasi dengan score
score = grid.score(X,y)

print(f'Hasil evaluasi: {score}')
print(f'Konfigurasi hyperparameter: {grid.best_params_}')

c:\Users\Administrator\anaconda3\lib\site-packages\sklearn\model_selection\_validation.py:372: FitFailedWarning: 
72 fits failed out of a total of 216.
The score on these train-test partitions for these parameters will be set to nan.
If these failures are not expected, you can try to debug them by setting error_score='raise'.

Below are more details about the failures:
--------------------------------------------------------------------------------
72 fits failed with the following error:
Traceback (most recent call last):
  File "c:\Users\Administrator\anaconda3\lib\site-packages\sklearn\model_selection\_validation.py", line 680, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "c:\Users\Administrator\anaconda3\lib\site-packages\sklearn\tree\_classes.py", line 937, in fit
    super().fit(
  File "c:\Users\Administrator\anaconda3\lib\site-packages\sklearn\tree\_classes.py", line 352, in fit
    criterion = CRITERIA_CLF[self.criterion](
KeyError: 'log_loss'

  

933 ms ± 96.7 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)
Hasil evaluasi: 1.0
Konfigurasi hyperparameter: {'criterion': 'gini', 'max_depth': 8}


c:\Users\Administrator\anaconda3\lib\site-packages\sklearn\model_selection\_validation.py:372: FitFailedWarning: 
72 fits failed out of a total of 216.
The score on these train-test partitions for these parameters will be set to nan.
If these failures are not expected, you can try to debug them by setting error_score='raise'.

Below are more details about the failures:
--------------------------------------------------------------------------------
72 fits failed with the following error:
Traceback (most recent call last):
  File "c:\Users\Administrator\anaconda3\lib\site-packages\sklearn\model_selection\_validation.py", line 680, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "c:\Users\Administrator\anaconda3\lib\site-packages\sklearn\tree\_classes.py", line 937, in fit
    super().fit(
  File "c:\Users\Administrator\anaconda3\lib\site-packages\sklearn\tree\_classes.py", line 352, in fit
    criterion = CRITERIA_CLF[self.criterion](
KeyError: 'log_loss'

  